# 2 Bike Data Preparation 

### Dublin Bike CA 1



Ronan Downes December 2022 

Prerequisite Notebook: **1_Load**

Runtime about 4 minutes
***

In [1]:
# ## Might get annoying so clear current output if required
# from IPython.display import Image
# Image(filename =r'bike2.gif', width = 600, height = 300)

## Import libraries 

In [2]:
#import necessary libraries and files 
import pandas as pd
import numpy as np
import warnings
import time
import datetime as dt
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import folium
import sklearn
import seaborn as sns
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
%matplotlib inline


## Import local Data

In [3]:

df4=  pd.read_csv('data/2021_Q4.csv')  ### For faster testing
df = pd.read_csv('data/01_Loaded_Bikes.csv')
dfw=pd.read_csv('data/01_Loaded_Weather.csv')
dft = pd.read_csv("data/01_Loaded_Travel_2006.csv")  
dfT = pd.read_csv("data/01_Loaded_Travel_2011.csv")


### Rough look at Bike Data

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11283524 entries, 0 to 11283523
Data columns (total 11 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   STATION ID             int64  
 1   TIME                   object 
 2   LAST UPDATED           object 
 3   NAME                   object 
 4   BIKE STANDS            int64  
 5   AVAILABLE BIKE STANDS  int64  
 6   AVAILABLE BIKES        int64  
 7   STATUS                 object 
 8   ADDRESS                object 
 9   LATITUDE               float64
 10  LONGITUDE              float64
dtypes: float64(2), int64(4), object(5)
memory usage: 947.0+ MB


In [5]:
df.nunique (axis=0, dropna=True)

STATION ID                   111
TIME                      103038
LAST UPDATED             5039168
NAME                         111
BIKE STANDS                   18
AVAILABLE BIKE STANDS         41
AVAILABLE BIKES               41
STATUS                         2
ADDRESS                      111
LATITUDE                     111
LONGITUDE                    111
dtype: int64

In [6]:
df.isnull().sum()   # Bike data has no missing data

STATION ID               0
TIME                     0
LAST UPDATED             0
NAME                     0
BIKE STANDS              0
AVAILABLE BIKE STANDS    0
AVAILABLE BIKES          0
STATUS                   0
ADDRESS                  0
LATITUDE                 0
LONGITUDE                0
dtype: int64

### Cleaning and filtering bike data

In [7]:
df.shape

(11283524, 11)

In [8]:
#Let dfc mean dataframe containing closed stations
df = df[df['STATUS'] == 'Open']
df.shape

(11283165, 11)

In [9]:
print (11189170-11188811, "rows of closed stations removed")

359 rows of closed stations removed


In [10]:
duplicate_rows_df = df[df.duplicated()]
print("number of duplicate rows: ", duplicate_rows_df.shape)


number of duplicate rows:  (94354, 11)


In [11]:
df = df.drop_duplicates()
duplicate_rows_df = df[df.duplicated()]
print("number of duplicate rows: ", duplicate_rows_df.shape)

number of duplicate rows:  (0, 11)


###  Filter to available post COVID-19 lockdoown  dates and removes duplicate rows
Usage of the DataFrame.loc[] Method to Filter Data to interval of interest and drop the "TIME" feature because it is reduntant.

The aim is to plan rebalancing and growth based on ML models so COVID-19 lockdown and xmas Holidays are ommitted.
Memory usage is 685 MB after Date filter and before merging weather data.

In [12]:
start_date = '2021-04-12'    #Lockdown restrictions lifted
end_date = '2021-12-11'      # Traditiona date for beginning of xmas holidays 
after_start_date = df['LAST UPDATED'] >= start_date
before_end_date = df['LAST UPDATED'] <= end_date
between_two_dates = after_start_date & before_end_date
# Using pandas.DataFrame.loc to Filter Rows by Dates
df = df.loc[between_two_dates]
df.drop_duplicates(keep= 'first',inplace=True)
df.info()   #memory usage: 319.6+ MB
df.shape # shape is (4188876, 9)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7413651 entries, 3109999 to 11283523
Data columns (total 11 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   STATION ID             int64  
 1   TIME                   object 
 2   LAST UPDATED           object 
 3   NAME                   object 
 4   BIKE STANDS            int64  
 5   AVAILABLE BIKE STANDS  int64  
 6   AVAILABLE BIKES        int64  
 7   STATUS                 object 
 8   ADDRESS                object 
 9   LATITUDE               float64
 10  LONGITUDE              float64
dtypes: float64(2), int64(4), object(5)
memory usage: 678.7+ MB


(7413651, 11)

In [13]:
#     dfs.columns = dfs.columns.str.replace(' ','_')
#     dfs.columns = dfs.columns.str.lower()
#     dfs.columns = dfs.columns.str.capitalize()
#     return




df.drop(['STATUS','TIME'], axis=1, inplace=True)  # Status is always open so is dropped 
# and at The temporal resolution of this study "time" CAN BE DROPPED
df.drop_duplicates(keep= 'first',inplace=True)    
#remove rows where no bike has been taken or returned since previous readings






In [14]:
df.head()

,STATION ID,LAST UPDATED,NAME,BIKE STANDS,AVAILABLE BIKE STANDS,AVAILABLE BIKES,ADDRESS,LATITUDE,LONGITUDE
3109999,2,2021-04-12 00:00:15,BLESSINGTON STREET,20,8,12,Blessington Street,53.35677,-6.26814
3110001,2,2021-04-12 00:10:21,BLESSINGTON STREET,20,8,12,Blessington Street,53.35677,-6.26814
3110003,2,2021-04-12 00:20:28,BLESSINGTON STREET,20,8,12,Blessington Street,53.35677,-6.26814
3110005,2,2021-04-12 00:30:34,BLESSINGTON STREET,20,8,12,Blessington Street,53.35677,-6.26814
3110007,2,2021-04-12 00:40:40,BLESSINGTON STREET,20,8,12,Blessington Street,53.35677,-6.26814


In [15]:
# User-defined functions to make data read and look better


def headers (dfs):
    dfs.columns = dfs.columns.str.replace(' ','_')
    dfs.columns = dfs.columns.str.lower()
    dfs.columns = dfs.columns.str.capitalize()
    return

# df.columns = df.columns.str.replace(' ','_')
# df.columns = df.columns.str.lower()
# df.columns = df.columns.str.capitalize()
# q4df.head()

In [16]:
headers(df)
df.head()

,Station_id,Last_updated,Name,Bike_stands,Available_bike_stands,Available_bikes,Address,Latitude,Longitude
3109999,2,2021-04-12 00:00:15,BLESSINGTON STREET,20,8,12,Blessington Street,53.35677,-6.26814
3110001,2,2021-04-12 00:10:21,BLESSINGTON STREET,20,8,12,Blessington Street,53.35677,-6.26814
3110003,2,2021-04-12 00:20:28,BLESSINGTON STREET,20,8,12,Blessington Street,53.35677,-6.26814
3110005,2,2021-04-12 00:30:34,BLESSINGTON STREET,20,8,12,Blessington Street,53.35677,-6.26814
3110007,2,2021-04-12 00:40:40,BLESSINGTON STREET,20,8,12,Blessington Street,53.35677,-6.26814


In [17]:


df.rename(columns={'Station_id':'Id',"Bike_stands": "Total", "Available_bike_stands": "Docks","Available_bikes":"Bikes"}, inplace = True)
df.sample(11)


,Id,Last_updated,Name,Total,Docks,Bikes,Address,Latitude,Longitude
10262576,4,2021-12-01 00:57:13,GREEK STREET,20,16,4,Greek Street,53.346874,-6.272976
5426428,107,2021-06-24 06:29:02,CHARLEVILLE ROAD,40,27,13,Charleville Road,53.359158,-6.281866
6424828,37,2021-07-27 14:34:58,ST. STEPHEN'S GREEN SOUTH,30,22,8,St. Stephen's Green South,53.337494,-6.261990
4017196,110,2021-05-10 09:29:48,PHIBSBOROUGH ROAD,40,38,2,Phibsborough Road,53.356308,-6.273717
10246432,73,2021-11-28 17:18:51,FRANCIS STREET,30,19,11,Francis Street,53.342079,-6.275233
5572263,66,2021-06-29 15:22:58,NEW CENTRAL BANK,40,33,7,New Central Bank,53.347122,-6.234749
5199763,83,2021-06-17 05:43:55,EMMET ROAD,40,4,36,Emmet Road,53.340714,-6.308191
7451294,3,2021-08-29 18:49:30,BOLTON STREET,20,17,3,Bolton Street,53.351181,-6.269859
10225769,113,2021-11-27 23:24:01,MERRION SQUARE SOUTH,40,26,14,Merrion Square South,53.338615,-6.248606
6336336,54,2021-07-24 16:58:34,CLONMEL STREET,33,24,9,Clonmel Street,53.336021,-6.262980


In [18]:
#Splitting "Last_updated"-Unique New Headings by going back to caps
df['DATETIME'] = [dt.datetime.strptime(d, "%Y-%m-%d %H:%M:%S") for d in df["Last_updated"]]
df['Last_updated'] = [dt.datetime.time(d) for d in df['DATETIME']] 
df['DATE'] = [dt.datetime.date(d) for d in df['DATETIME']] 
df['date_for_merge'] = df['DATETIME'].dt.round('H')

In [19]:
df.sample(11)

,Id,Last_updated,Name,Total,Docks,Bikes,Address,Latitude,Longitude,DATETIME,DATE,date_for_merge
9800076,2,19:32:06,BLESSINGTON STREET,20,20,0,Blessington Street,53.356770,-6.268140,2021-11-13 19:32:06,2021-11-13,2021-11-13 20:00:00
3910838,66,02:23:09,NEW CENTRAL BANK,40,23,16,New Central Bank,53.347122,-6.234749,2021-05-07 02:23:09,2021-05-07,2021-05-07 02:00:00
8542344,13,13:50:27,FITZWILLIAM SQUARE WEST,30,21,9,Fitzwilliam Square West,53.336075,-6.252825,2021-10-02 13:50:27,2021-10-02,2021-10-02 14:00:00
4072520,84,11:54:27,BROOKFIELD ROAD,30,14,16,Brookfield Road,53.339005,-6.300217,2021-05-12 11:54:27,2021-05-12,2021-05-12 12:00:00
9667179,34,18:04:56,PORTOBELLO HARBOUR,30,7,22,Portobello Harbour,53.330360,-6.265163,2021-11-07 18:04:56,2021-11-07,2021-11-07 18:00:00
3321276,90,20:56:49,BENSON STREET,40,28,12,Benson Street,53.344154,-6.233451,2021-04-18 20:56:49,2021-04-18,2021-04-18 21:00:00
8837176,97,02:31:50,KILMAINHAM GAOL,40,28,12,Kilmainham Gaol,53.342113,-6.310015,2021-10-11 02:31:50,2021-10-11,2021-10-11 03:00:00
4404830,34,17:25:10,PORTOBELLO HARBOUR,30,21,9,Portobello Harbour,53.330360,-6.265163,2021-05-23 17:25:10,2021-05-23,2021-05-23 17:00:00
9464327,102,21:09:32,WESTERN WAY,40,23,17,Western Way,53.354931,-6.269425,2021-10-31 21:09:32,2021-10-31,2021-10-31 21:00:00
6135045,36,20:20:23,ST. STEPHEN'S GREEN EAST,40,31,9,St. Stephen's Green East,53.337826,-6.256035,2021-07-17 20:20:23,2021-07-17,2021-07-17 20:00:00


In [20]:
#Bike Feature Engineering
df['Occupy_Pct'] = df['Bikes'] / df['Total']
df['Saturated'] = np.where(df['Occupy_Pct'] == 0, 1,0 )
df['Empty'] = np.where(df['Occupy_Pct'] == 1, 1,0 )


In [21]:
df.sample(33)

,Id,Last_updated,Name,Total,Docks,Bikes,Address,Latitude,Longitude,DATETIME,DATE,date_for_merge,Occupy_Pct,Saturated,Empty
4999692,32,15:02:48,PEARSE STREET,30,7,23,Pearse Street,53.344303,-6.250427,2021-06-11 15:02:48,2021-06-11,2021-06-11 15:00:00,0.766667,0,0
8245056,15,11:21:06,HARDWICKE STREET,16,12,4,Hardwicke Street,53.355473,-6.264423,2021-09-23 11:21:06,2021-09-23,2021-09-23 11:00:00,0.250000,0,0
4993661,9,16:28:15,EXCHEQUER STREET,24,5,19,Exchequer Street,53.343033,-6.263578,2021-06-11 16:28:15,2021-06-11,2021-06-11 16:00:00,0.791667,0,0
4435609,32,14:24:26,PEARSE STREET,30,22,8,Pearse Street,53.344303,-6.250427,2021-05-24 14:24:26,2021-05-24,2021-05-24 14:00:00,0.266667,0,0
4157727,50,17:31:43,GEORGES LANE,40,32,8,George's Lane,53.350231,-6.279696,2021-05-15 17:31:43,2021-05-15,2021-05-15 18:00:00,0.200000,0,0
8755925,112,05:22:31,NORTH CIRCULAR ROAD (O'CONNELL'S),30,8,22,North Circular Road (O'Connell's),53.357841,-6.251557,2021-10-08 05:22:31,2021-10-08,2021-10-08 05:00:00,0.733333,0,0
4928264,116,14:43:04,BROADSTONE,30,21,9,Broadstone,53.354698,-6.272314,2021-06-08 14:43:04,2021-06-08,2021-06-08 15:00:00,0.300000,0,0
10139714,33,04:06:47,PRINCES STREET / O'CONNELL STREET,23,15,8,Princes Street / O'Connell Street,53.349014,-6.260311,2021-11-25 04:06:47,2021-11-25,2021-11-25 04:00:00,0.347826,0,0
9516903,63,10:29:39,FENIAN STREET,35,7,28,Fenian Street,53.341427,-6.246720,2021-11-02 10:29:39,2021-11-02,2021-11-02 10:00:00,0.800000,0,0
5145631,113,06:42:10,MERRION SQUARE SOUTH,40,34,6,Merrion Square South,53.338615,-6.248606,2021-06-15 06:42:10,2021-06-15,2021-06-15 07:00:00,0.150000,0,0


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4188870 entries, 3109999 to 10611332
Data columns (total 15 columns):
 #   Column          Dtype         
---  ------          -----         
 0   Id              int64         
 1   Last_updated    object        
 2   Name            object        
 3   Total           int64         
 4   Docks           int64         
 5   Bikes           int64         
 6   Address         object        
 7   Latitude        float64       
 8   Longitude       float64       
 9   DATETIME        datetime64[ns]
 10  DATE            object        
 11  date_for_merge  datetime64[ns]
 12  Occupy_Pct      float64       
 13  Saturated       int32         
 14  Empty           int32         
dtypes: datetime64[ns](2), float64(3), int32(2), int64(4), object(4)
memory usage: 479.4+ MB


In [23]:
#remove rows where no bike has been taken or returned since previous readings
df.drop_duplicates(keep= 'first',inplace=True)

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4188870 entries, 3109999 to 10611332
Data columns (total 15 columns):
 #   Column          Dtype         
---  ------          -----         
 0   Id              int64         
 1   Last_updated    object        
 2   Name            object        
 3   Total           int64         
 4   Docks           int64         
 5   Bikes           int64         
 6   Address         object        
 7   Latitude        float64       
 8   Longitude       float64       
 9   DATETIME        datetime64[ns]
 10  DATE            object        
 11  date_for_merge  datetime64[ns]
 12  Occupy_Pct      float64       
 13  Saturated       int32         
 14  Empty           int32         
dtypes: datetime64[ns](2), float64(3), int32(2), int64(4), object(4)
memory usage: 479.4+ MB


In [25]:
# Day_N is in mod 7 and maps directly to days of the week

df['Day_N'] = df.DATETIME.dt.dayofweek
df['Day_type'] = np.where(df['Day_N'] <= 4, 'Weekday', (np.where(df['Day_N'] == 5, 'Saturday', 'Sunday')))

def Hist_time(x):
    if x.time() < dt.time(6):
        return "Overnight "
    elif x.time() < dt.time(11):
        return "6 AM-10 AM "
    elif x.time() < dt.time(16):
        return "11 AM-3 PM "
    elif x.time() < dt.time(20):
        return "4 PM-7 PM "
    elif x.time() <= dt.time(23):
        return "8 PM-11 PM "
    else:
        return "Overnight "


df["Time_type"] = df['DATETIME'].apply(Hist_time)
df['Hour'] = df['DATETIME'].dt.hour
df['Month'] = df['DATETIME'].dt.month
df['Bike_cluster'] = df['Time_type'] + df['Day_type']

df.sample(5)

,Id,Last_updated,Name,Total,Docks,Bikes,Address,Latitude,Longitude,DATETIME,...,date_for_merge,Occupy_Pct,Saturated,Empty,Day_N,Day_type,Time_type,Hour,Month,Bike_cluster
10166338,12,14:46:33,ECCLES STREET,20,10,10,Eccles Street,53.359245,-6.269779,2021-11-26 14:46:33,...,2021-11-26 15:00:00,0.500000,0,0,4,Weekday,11 AM-3 PM,14,11,11 AM-3 PM Weekday
7885861,88,16:42:24,BLACKHALL PLACE,30,1,29,Blackhall Place,53.348801,-6.281637,2021-09-11 16:42:24,...,2021-09-11 17:00:00,0.966667,0,0,5,Saturday,4 PM-7 PM,16,9,4 PM-7 PM Saturday
7134785,4,19:03:52,GREEK STREET,20,10,10,Greek Street,53.346874,-6.272976,2021-08-19 19:03:52,...,2021-08-19 19:00:00,0.500000,0,0,3,Weekday,4 PM-7 PM,19,8,4 PM-7 PM Weekday
3134481,94,00:10:24,HEUSTON STATION (CAR PARK),40,34,6,Heuston Station (Car Park),53.346985,-6.297804,2021-04-12 00:10:24,...,2021-04-12 00:00:00,0.150000,0,0,0,Weekday,Overnight,0,4,Overnight Weekday
9735651,111,00:53:54,MOUNTJOY SQUARE EAST,40,8,32,Mountjoy Square East,53.356716,-6.256359,2021-11-09 00:53:54,...,2021-11-09 01:00:00,0.800000,0,0,1,Weekday,Overnight,0,11,Overnight Weekday


In [26]:
df.rename(columns={'DATETIME':'Datetime','DATE':'date'}, inplace = True)
df.sample(11)


,Id,Last_updated,Name,Total,Docks,Bikes,Address,Latitude,Longitude,Datetime,...,date_for_merge,Occupy_Pct,Saturated,Empty,Day_N,Day_type,Time_type,Hour,Month,Bike_cluster
8207822,111,04:00:02,MOUNTJOY SQUARE EAST,40,20,20,Mountjoy Square East,53.356716,-6.256359,2021-09-21 04:00:02,...,2021-09-21 04:00:00,0.500000,0,0,1,Weekday,Overnight,4,9,Overnight Weekday
5340661,22,11:13:57,TOWNSEND STREET,20,6,14,Townsend Street,53.345921,-6.254614,2021-06-22 11:13:57,...,2021-06-22 11:00:00,0.700000,0,0,1,Weekday,11 AM-3 PM,11,6,11 AM-3 PM Weekday
6672322,27,06:12:54,MOLESWORTH STREET,20,16,4,Molesworth Street,53.341290,-6.258117,2021-08-04 06:12:54,...,2021-08-04 06:00:00,0.200000,0,0,2,Weekday,6 AM-10 AM,6,8,6 AM-10 AM Weekday
10312268,73,17:57:32,FRANCIS STREET,30,13,17,Francis Street,53.342079,-6.275233,2021-12-02 17:57:32,...,2021-12-02 18:00:00,0.566667,0,0,3,Weekday,4 PM-7 PM,17,12,4 PM-7 PM Weekday
7637790,107,08:06:15,CHARLEVILLE ROAD,40,27,12,Charleville Road,53.359158,-6.281866,2021-09-03 08:06:15,...,2021-09-03 08:00:00,0.300000,0,0,4,Weekday,6 AM-10 AM,8,9,6 AM-10 AM Weekday
10508335,91,08:53:20,SOUTH DOCK ROAD,30,20,10,South Dock Road,53.341831,-6.231291,2021-12-08 08:53:20,...,2021-12-08 09:00:00,0.333333,0,0,2,Weekday,6 AM-10 AM,8,12,6 AM-10 AM Weekday
3134744,94,22:07:39,HEUSTON STATION (CAR PARK),40,38,2,Heuston Station (Car Park),53.346985,-6.297804,2021-04-12 22:07:39,...,2021-04-12 22:00:00,0.050000,0,0,0,Weekday,8 PM-11 PM,22,4,8 PM-11 PM Weekday
3583058,12,23:21:47,ECCLES STREET,20,15,5,Eccles Street,53.359245,-6.269779,2021-04-27 23:21:47,...,2021-04-27 23:00:00,0.250000,0,0,1,Weekday,Overnight,23,4,Overnight Weekday
10226132,115,05:36:24,KILLARNEY STREET,30,13,17,Killarney Street,53.354843,-6.247579,2021-11-27 05:36:24,...,2021-11-27 06:00:00,0.566667,0,0,5,Saturday,Overnight,5,11,Overnight Saturday
5342593,29,04:13:34,ORMOND QUAY UPPER,29,24,5,Ormond Quay Upper,53.346058,-6.268001,2021-06-22 04:13:34,...,2021-06-22 04:00:00,0.172414,0,0,1,Weekday,Overnight,4,6,Overnight Weekday


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4188870 entries, 3109999 to 10611332
Data columns (total 21 columns):
 #   Column          Dtype         
---  ------          -----         
 0   Id              int64         
 1   Last_updated    object        
 2   Name            object        
 3   Total           int64         
 4   Docks           int64         
 5   Bikes           int64         
 6   Address         object        
 7   Latitude        float64       
 8   Longitude       float64       
 9   Datetime        datetime64[ns]
 10  date            object        
 11  date_for_merge  datetime64[ns]
 12  Occupy_Pct      float64       
 13  Saturated       int32         
 14  Empty           int32         
 15  Day_N           int64         
 16  Day_type        object        
 17  Time_type       object        
 18  Hour            int64         
 19  Month           int64         
 20  Bike_cluster    object        
dtypes: datetime64[ns](2), float64(3), int32(2), int

In [28]:
df.to_csv("data/02_Bike_Rich.csv", index=False)